In [1]:
%run "00_wall_of_imports.ipynb"

2025-02-25 03:32:41 - PyTorch version 2.6.0 available.


C:\Users\dabra\AppData\Roaming\Python\Python313\site-packages\pydantic\v1\typing.py:68: DeprecationWarning: Failing to pass a value to the 'type_params' parameter of 'typing.ForwardRef._evaluate' is deprecated, as it leads to incorrect behaviour when calling typing.ForwardRef._evaluate on a stringified annotation that references a PEP 695 type parameter. It will be disallowed in Python 3.15.
  return cast(Any, type_)._evaluate(globalns, localns, recursive_guard=set())
C:\Users\dabra\AppData\Roaming\Python\Python313\site-packages\pydantic\v1\typing.py:68: DeprecationWarning: Failing to pass a value to the 'type_params' parameter of 'typing.ForwardRef._evaluate' is deprecated, as it leads to incorrect behaviour when calling typing.ForwardRef._evaluate on a stringified annotation that references a PEP 695 type parameter. It will be disallowed in Python 3.15.
  return cast(Any, type_)._evaluate(globalns, localns, recursive_guard=set())
C:\Users\dabra\AppData\Roaming\Python\Python313\site-p

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key:")

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.7)

embeddings = OpenAIEmbeddings()

C:\Users\dabra\AppData\Local\Temp\ipykernel_21660\1919260345.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.7)


In [ ]:
# Load the vector store
travel_db = FAISS.load_local("travel_db_faiss", embeddings, allow_dangerous_deserialization=True)

# Load the docstore
with open('travel_db_docstore.pkl', 'rb') as f:
    travel_db.docstore = pickle.load(f)

# Verify it loaded correctly
print(f"Loaded knowledge base with {len(travel_db.index_to_docstore_id)} documents")

In [4]:
# Define the AgentState class as in the notebook
class AgentState(BaseModel):
    """State for the travel assistant workflow."""

    query: str
    chat_history: List[Any] = Field(default_factory=list)
    agent_executor: Optional[str] = None
    agent_response: Optional[str] = None
    final_response: Optional[str] = None
    context: Dict[str, Any] = Field(default_factory=dict)
    error: Optional[str] = None
    messages: List[BaseMessage] = Field(default_factory=list)
    
    def __repr__(self):
         # Defines string representation of the AgentState object
        return f"AgentState(query={self.query}, agent_executor={self.agent_executor})"
        
    def to_dict(self):
        """Convert state to dictionary for LangGraph compatibility"""
        # Method to convert the state object to a dictionary
        return {
            "query": self.query,
            "chat_history": self.chat_history,
            "agent_executor": self.agent_executor,
            "agent_response": self.agent_response,
            "final_response": self.final_response,
            "context": self.context,
            "error": self.error,
            "messages": self.messages
        }

In [5]:
# Load OpenAI API key
@cl.on_chat_start
async def on_chat_start():
    # Initialize the LLM
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.7)
    
    # Load the vector store
    try:
        travel_db = FAISS.load_local("travel_db_faiss", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
        
        # Load the docstore
        with open('travel_db_docstore.pkl', 'rb') as f:
            travel_db.docstore = pickle.load(f)
            
        print(f"Loaded knowledge base with {len(travel_db.index_to_docstore_id)} documents")
    except Exception as e:
        print(f"Error loading knowledge base: {e}")
        travel_db = None

    # Define the router agent function
    def router_agent(state: AgentState) -> dict:
        """Router agent that determines which specialized agent should handle the query."""

        # Create a chat prompt template with system and human messages
        router_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a travel assistant router. Your job is to determine which specialized agent
            should handle the user's travel-related query. Choose the most appropriate agent from:
            
            - itinerary_agent: For requests to create travel itineraries, vacation plans, or multi-day travel schedules
            - flight_agent: For questions about flights, airfares, airlines, or flight bookings
            - accommodation_agent: For questions about hotels, resorts, accommodations, or places to stay
            - information_agent: For general travel information, destination facts, or travel advice
            
            Respond ONLY with the name of the appropriate agent. Do not include any explanations or additional text.
            """),
            ("human", "{query}")
        ])
        
        # Use a chain to pass the user query to the LLM, get the recommended agent name and parse the output as a string
        chain = router_prompt | llm | (lambda x: x.content)

        agent_executor = chain.invoke({"query": state.query}).strip()
        
        valid_agents = ["itinerary_agent", "flight_agent", "accommodation_agent", "information_agent"]

        # Validate the response
        if agent_executor not in valid_agents:
            # Default to information agent if invalid response
            agent_executor = "information_agent"
                
        # Return state as a dictionary
        return {"agent_executor": agent_executor, 
                "query": state.query,
                "chat_history": state.chat_history,
                "agent_response": state.agent_response,
                "final_response": state.final_response,
                "context": state.context,
                "error": state.error}

    # Setup RAG chain for information retrieval
    def setup_rag_chain():
        """Set up the RAG chain for information retrieval."""
        # Create a retriever from our travel knowledge base
        if travel_db is not None:
            retriever = travel_db.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 7}
            )
                
            rag_prompt = ChatPromptTemplate.from_template("""You are a knowledgeable travel assistant with expertise in destinations worldwide.
                Use the following travel information to provide detailed, accurate responses to the user's query.
                If the retrieved information doesn't fully answer the question, use your knowledge to provide
                the best possible response, but prioritize the retrieved information.
                
                Retrieved information: {context}        
                Question: {input}
            """)

            # Create the document processing chain
            document_chain = create_stuff_documents_chain(llm, rag_prompt)
            
            return create_retrieval_chain(retriever, document_chain)
        else:
            # Return None if vector store is not available
            return None

    # Define specialized agents
    def itinerary_agent(state: AgentState) -> dict:
        """Creates customized travel itineraries based on user preferences."""
        try:
            # First, retrieve relevant destination information
            rag_chain = setup_rag_chain()
            if rag_chain:
                retrieval_result = rag_chain.invoke({"input": state.query})
            else:
                retrieval_result = {"answer": "I don't have specific information about this destination, but I'll do my best to help."}
            
            # Extract destination information from query using LLM
            extraction_prompt = ChatPromptTemplate.from_template("""Extract the key travel information from the user's query.
            Return a JSON object with these fields (leave empty if not mentioned):
            {{
                "destinations": ["list of mentioned destinations"],
                "duration": "total trip duration in days",
                "budget": "budget information",
                "interests": ["list of mentioned interests/activities"],
                "travel_dates": "approximate travel dates",
                "travelers": "number and type of travelers (family, couple, solo, etc.)"
            }}
            
            Query: {input}
            """)       
            
            extraction_chain = extraction_prompt | llm | (lambda x: x.content)

            try:
                # First check if the string is not empty
                extraction_result = extraction_chain.invoke({"input": state.query}).strip()
                if extraction_result:
                    extracted_info = json.loads(extraction_result)
                else:
                    extracted_info = {
                        "destinations": [],
                        "duration": "",
                        "budget": "",
                        "interests": [],
                        "travel_dates": "",
                        "travelers": ""
                    }
                state.context.update(extracted_info)
            except json.JSONDecodeError:
                # Provide default structure when parsing fails
                extracted_info = {
                    "destinations": [],
                    "duration": "",
                    "budget": "",
                    "interests": [],
                    "travel_dates": "",
                    "travelers": ""
                }
                state.context.update(extracted_info)

            
            # Generate itinerary using retrieved information and extracted parameters
            itinerary_prompt = ChatPromptTemplate.from_template("""You are a travel itinerary expert. Create a detailed day-by-day travel itinerary
            based on the user's preferences and the retrieved destination information.
            
            For each day, include morning activities, lunch suggestions, afternoon activities, dinner recommendations, and 
            evening activities or relaxation options
            
            Also include practical advice about transportation between attractions, estimated costs, time management tips, and local customs.
            
            Make the itinerary realistic in terms of travel times and activities per day.
            
            Context information:
            {context_str}
            
            Extracted travel parameters:
            {parameters}
            
            Query: {input}
            """)        
            
            # Format the context and parameters for the prompt
            context_str = retrieval_result.get("answer", "")
            parameters_str = json.dumps(state.context, indent=2)
            
            itinerary_chain = itinerary_prompt | llm | (lambda x: x.content)
            response = itinerary_chain.invoke({
                "input": state.query,
                "context_str": context_str,
                "parameters": parameters_str
            })
            
            return {
                "agent_response": response,
                "context": state.context,
                "query": state.query
            }
            
        except Exception as e:
            return {
                "error": str(e),
                "context": state.context,
                "query": state.query
            }

    def flight_agent(state: AgentState) -> dict:
        """Handles flight-related questions and searches."""
        try:
            # Extract flight search parameters
            extraction_prompt = ChatPromptTemplate.from_template("""Extract flight search parameters from the user's query.
                Return a JSON object with these fields (leave empty if not mentioned):
            {{
                "origin": "origin airport or city code",
                "destination": "destination airport or city code",
                "departure_date": "departure date in YYYY-MM-DD format",
                "return_date": "return date in YYYY-MM-DD format (if round-trip)",
                "num_passengers": "number of passengers",
                "cabin_class": "economy/business/first",
                "price_range": "budget constraints",
                "airline_preferences": ["preferred airlines"]
            }}
            
            Query: {input}
            """)       
            
            extraction_chain = extraction_prompt | llm | (lambda x: x.content)

            try:
                flight_params = json.loads(extraction_chain.invoke({"input": state.query}))
                state.context.update({"flight_params": flight_params})
            except json.JSONDecodeError:
                state.context.update({"flight_params": {}})
            
            # Get flight information using RAG
            rag_chain = setup_rag_chain()
            if rag_chain:
                retrieval_result = rag_chain.invoke({"input": state.query})
            else:
                retrieval_result = {"answer": "I don't have specific flight information in my database, but I can provide general advice."}

            flight_prompt = ChatPromptTemplate.from_template("""You are a flight search specialist. Provide helpful information about flights
                based on the retrieved flight data and the user's query. Include details about available flights matching the criteria, 
                price ranges and fare comparisons, airline options, departure/arrival times, travel duration, layovers (if applicable), 
                and booking recommendations.
                
                If exact flight information isn't available in the retrieved data, provide general advice
                about the requested route, typical prices, and best booking strategies.
                
                Retrieved flight information:
                {context_str}
                
                Extracted flight parameters:
                {parameters}
            
                Query: {input}
                """)
            
            # Format the context and parameters for the prompt
            context_str = retrieval_result.get("answer", "")
            parameters_str = json.dumps(state.context.get("flight_params", {}), indent=2)
            
            flight_chain = flight_prompt | llm | (lambda x: x.content)
            response = flight_chain.invoke({
                "input": state.query,
                "context_str": context_str,
                "parameters": parameters_str
            })
            
            return {
                "agent_response": response,
                "context": state.context,
                "query": state.query
            }
            
        except Exception as e:
            return {
                "error": str(e),
                "context": state.context,
                "query": state.query
            }

    def accommodation_agent(state: AgentState) -> dict:
        """Provides hotel and accommodation recommendations."""
        try:
            # Extract accommodation preferences
            extraction_prompt = ChatPromptTemplate.from_messages([
                ("system", """Extract accommodation preferences from the user's query.
                Return a JSON object with these fields (leave empty if not mentioned):
                {{
                    "location": "city or specific area",
                    "check_in_date": "in YYYY-MM-DD format",
                    "check_out_date": "in YYYY-MM-DD format",
                    "guests": "number of guests",
                    "rooms": "number of rooms",
                    "budget_range": "price range per night"
                }}
                """),
                ("human", "{query}")
            ])
            
            extraction_chain = extraction_prompt | llm | (lambda x: x.content)
            try:
                accommodation_params = json.loads(extraction_chain.invoke({"query": state.query}))
                state.context.update({"accommodation_params": accommodation_params})
            except json.JSONDecodeError:
                state.context.update({"accommodation_params": {}})
            
            # Get accommodation information using RAG
            rag_chain = setup_rag_chain()
            if rag_chain:
                retrieval_result = rag_chain.invoke({"input": f"hotels in {state.context.get('accommodation_params', {}).get('location', '')}"})
            else:
                retrieval_result = {"context": "I don't have specific accommodation information in my database, but I can provide general advice."}
            
            # Generate response based on accommodation parameters and retrieved information
            accommodation_prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a hotel and accommodation expert. Provide detailed recommendations
                based on the user's preferences and the retrieved accommodation data. Include:
                
                - Suitable hotel/accommodation options
                - Price ranges and value considerations
                - Location benefits and proximity to attractions
                - Amenities and facilities
                - Guest ratings and reviews summary
                - Booking tips and optimal timing
                
                If specific accommodation data isn't available, provide general advice about
                accommodations in the requested location, typical options at different price points,
                and best areas to stay.
                
                Retrieved accommodation information:
                {context}
                
                Extracted accommodation parameters:
                {accommodation_params}
                """),
                ("human", "{query}")
            ])
            
            accommodation_chain = accommodation_prompt | llm | (lambda x: x.content)
            response = accommodation_chain.invoke({
                "query": state.query,
                "context": retrieval_result.get("context", ""),
                "accommodation_params": json.dumps(state.context.get("accommodation_params", {}), indent=2)
            })
            
            return {"agent_response": response}
            
        except Exception as e:
            return {"error": str(e)}

    def information_agent(state: AgentState) -> dict:
        """Answers general travel questions using RAG."""
        try:
            # This agent directly uses the RAG chain to provide travel information
            rag_chain = setup_rag_chain()
            if rag_chain:
                result = rag_chain.invoke({"input": state.query})
            else:
                result = {"answer": "I don't have specific information about this in my travel database, but I'll provide general advice based on my knowledge."}
            
            # Enhance RAG response with additional context if needed
            enhancement_prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a knowledgeable travel information specialist. Review and enhance
                the retrieved information to provide a comprehensive, accurate response to the user's query.
                
                If the retrieved information is incomplete, add relevant details from your knowledge while
                clearly distinguishing between retrieved facts and general knowledge.
                
                Focus on providing practical, useful information that directly addresses the user's needs.
                Include cultural insights, traveler tips, and seasonal considerations when relevant.
                
                Retrieved information:
                {rag_response}
                """),
                ("human", "{input}")
            ])
            
            enhancement_chain = enhancement_prompt | llm | (lambda x: x.content)
            enhanced_response = enhancement_chain.invoke({
                "input": state.query,
                "rag_response": result.get("answer", "")
            })
            
            return {"agent_response": enhanced_response}
            
        except Exception as e:
            return {"error": str(e)}

    # Response Generator - Creates the final response
    def generate_final_response(state: AgentState) -> dict:
        """Generates the final, polished response to the user."""
        # Create a consistent, helpful response format
        formatting_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a friendly, helpful travel assistant. Format the specialized agent's response
            into a clear, well-structured, and engaging reply. Maintain all the factual information and advice
            while improving readability with:
            
            - A warm, conversational tone
            - Logical organization with headings where appropriate
            - Bullet points for lists
            - Bold text for important information
            - Emojis where appropriate (but not excessive)
            
            Make sure the response completely addresses the user's query. Add a brief, friendly closing
            that invites further questions.
            
            Original agent response:
            {agent_response}
            """),
            ("human", "{query}")
        ])
        
        formatting_chain = formatting_prompt | llm | (lambda x: x.content)
        final_response = formatting_chain.invoke({
            "query": state.query,
            "agent_response": state.agent_response
        })
        
        return {"agent_response": final_response}

    # Error Handler - Manages errors gracefully
    def handle_error(state: AgentState) -> dict:
        """Handles errors and provides a graceful fallback response."""
        error_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a helpful travel assistant. The system encountered an error while
            processing the user's query. Provide a helpful response that:
            
            1. Acknowledges the issue
            2. Offers general travel advice related to their query
            3. Suggests how they might rephrase their question for better results
            
            Error message: {error}
            """),
            ("human", "{query}")
        ])
        
        error_chain = error_prompt | llm | (lambda x: x.content)
        fallback_response = error_chain.invoke({
            "query": state.query,
            "error": state.error or "Unknown error occurred"
        })
        
        return {"agent_response": fallback_response}
    
    # Create the Travel Assistant graph
    workflow = StateGraph(AgentState)
    
    # Add nodes for each agent and processing step
    workflow.add_node("router", router_agent)
    workflow.add_node("itinerary_agent", itinerary_agent)
    workflow.add_node("flight_agent", flight_agent)
    workflow.add_node("accommodation_agent", accommodation_agent)
    workflow.add_node("information_agent", information_agent)
    workflow.add_node("response_generator", generate_final_response)
    workflow.add_node("error_handler", handle_error)
    
    # Define conditional edge routing
    def router_edges(state):
        if state.agent_executor == "itinerary_agent":
            return "itinerary_agent"
        elif state.agent_executor == "flight_agent":
            return "flight_agent"
        elif state.agent_executor == "accommodation_agent":
            return "accommodation_agent"
        else:
            return "information_agent"
    
    # Check if there's an error and route accordingly
    def agent_edges(state):
        if state.error is not None:
            return "error_handler"
        else:
            return "response_generator"
    
    # Set entry point
    workflow.set_entry_point("router")
    
    # Connect router to agents
    workflow.add_conditional_edges("router", router_edges)
    
    # Connect agents to next nodes
    for agent in ["itinerary_agent", "flight_agent", "accommodation_agent", "information_agent"]:
        workflow.add_conditional_edges(agent, agent_edges)
    
    # Connect to end
    workflow.add_edge("response_generator", END)
    workflow.add_edge("error_handler", END)
    
    # Compile the graph
    travel_assistant = workflow.compile()
    
    # Store the travel assistant in the user session
    cl.user_session.set("travel_assistant", travel_assistant)

    # Greet the user with an introduction message
    await cl.Message(
        content="👋 Hello! I'm your Travel Assistant. I can help with travel itineraries, flight information, accommodations, and general travel advice. How can I assist you today?",
    ).send()

In [6]:
@cl.on_message
async def on_message(message: cl.Message):
    # Get the travel assistant from the user session
    travel_assistant = cl.user_session.get("travel_assistant")
    
    # Get user's message
    user_query = message.content
    
    # Create a message to show the user that we're thinking
    thinking_msg = cl.Message(content="Thinking...")
    await thinking_msg.send()
    
    # Process the query through our travel assistant
    result = travel_assistant.invoke({
        "query": user_query,
        "chat_history": [],  # We could store and use chat history for context
    })
    
    # Get the final response
    response = result.get("agent_response", "I'm sorry, I couldn't process your request. Please try again.")
    
    # Update the thinking message with the actual response
    await thinking_msg.update(content=response)

In [ ]:
if __name__ == "__main__":
    # To run: chainlit run app.py -w
    pass